In [1]:
import pandas as pd
import numpy as np
import string as s
import seaborn as sns
import math
# import tables
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style, colors
from itertools import groupby
import matplotlib.patches as patches
from matplotlib import gridspec
import scipy.stats as stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, fclusterdata, leaves_list
from scipy.spatial.distance import pdist
from scipy.ndimage.interpolation import shift
# matplotlib.style.use('ggplot')
# %matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
# matplotlib.rcParams['figure.facecolor'] = 'white'
import mpl_toolkits.mplot3d.axes3d as p3
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets.samples_generator import make_swiss_roll
matplotlib.style.use('ggplot')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
wkdir = '/projects/da_workspace/DA-167_Cervical_epi_clustering/H3K4me3_promoter/'

In [3]:
# macs2 peaks intersected with promoter regions, require 10% overlap
f1 = '/projects/da_workspace/DA-167_Cervical_epi_clustering/H3K4me3_promoter/intersected_peaks.txt'

In [4]:
df1 =  pd.read_csv(f1, sep='\t', header=None)
df1.columns = ['peak_file']
df1.head(2)

,peak_file
0,HTMCP-03-06-02001.intersected
1,HTMCP-03-06-02006.intersected


In [39]:
dfm = pd.DataFrame()
for ix, f in df1.iterrows():
    fl = '{0}{1}'.format(wkdir, f.peak_file)
    pat = f.peak_file.split('.')[0]
    dftmp = pd.read_csv(fl, sep='\t', header=None, usecols=[0,1,2], low_memory=False)
    dftmp = dftmp.drop_duplicates()
    dftmp[pat] = 1
    dftmp['ix'] = dftmp.apply(lambda x: '_'.join([str(x[0]), str(x[1]), str(x[2])]), axis=1)
    dftmp = dftmp.drop([0,1,2], axis=1).set_index('ix')
    if dfm.empty:
        dfm = dftmp
    else:
        dfm = pd.merge(dfm, dftmp,left_index=True, right_index=True, how='outer' )


HTMCP-03-06-02001


(10672, 1)

HTMCP-03-06-02006


(12269, 2)

HTMCP-03-06-02007


...

(16155, 33)

HTMCP-03-06-02330


(16175, 34)

HTMCP-03-06-02332


(16180, 35)

In [41]:
dfm = dfm.fillna(0)
dfm.head()

,HTMCP-03-06-02001,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02012,HTMCP-03-06-02020,HTMCP-03-06-02026,HTMCP-03-06-02040,HTMCP-03-06-02042,HTMCP-03-06-02054,HTMCP-03-06-02058,...,HTMCP-03-06-02203,HTMCP-03-06-02205,HTMCP-03-06-02210,HTMCP-03-06-02214,HTMCP-03-06-02260,HTMCP-03-06-02266,HTMCP-03-06-02268,HTMCP-03-06-02320,HTMCP-03-06-02330,HTMCP-03-06-02332
ix,,,,,,,,,,,,,,,,,,,,,
10_100026007_100030006,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10_100172941_100176940,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
10_100204684_100208683,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
10_100993619_100997618,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10_101086857_101090856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
dfm.to_csv('{}H3K4me3.promoter.txt'.format(wkdir))